In [1]:
import pandas as pd
import numpy as np
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO   

In [5]:
f = "axis03.pdf"
tables=camelot.read_pdf(f, flavor="stream", pages="all")
if len(tables) !=0 :
    df=pd.DataFrame(); tmp=pd.DataFrame();
    for i in range(len(tables)):
        tmp=tables[i].df
        if (tmp.shape[1]==8)|(tmp.shape[1]==7)|(tmp.shape[1]==6)|(tmp.shape[1]==5):
            df=pd.concat([df,tmp]).drop_duplicates(keep="first").reset_index(drop=True)
        else:
            print(tmp.shape[1])
            print("\n different structure on page: ",i)

9

 different structure on page:  0
4

 different structure on page:  17
4

 different structure on page:  18


In [7]:
df.to_csv("xns.csv", index=0)

In [ ]:
import pandas as pd
import tabula
import camelot
import numpy as np

try:
    tables=camelot.read_pdf("axis05.pdf",flavor='stream' ,pages = "all")
             
    if len(tables) !=0 :
        df=pd.DataFrame();    tmp=pd.DataFrame(); 
        for i in range(len(tables)) :
            tmp=tables[i].df  
            if (tmp.shape[1] == 6 ):
                df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
            elif (tmp.shape[1] == 7):
                tmp.drop(tmp.columns[[1]], axis=1, inplace=True)
                tmp.columns = [0,1,2,3,4,5]
                df=pd.concat([df,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
            else:
                print(tmp.shape[1])
                print("\n different structure on page:",i); #log.write("\n different structure")        
except:
    pass

try:
    df.replace(to_replace = '', value =np.nan,inplace=True)
except:pass

In [ ]:
df.head()

In [ ]:
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
except:
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
    except:
        print("\nAxis-Column headers missing"); pass

In [ ]:
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass
df.head()

In [ ]:
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(), axis=1) ==True].index)]
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
df.head()

In [ ]:
try:
    narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
except:
    try:
        narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
        except:
            pass

In [ ]:
df = df[pd.notnull(df[narr])]

df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

In [ ]:
df.head()

In [ ]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
    df[bal]=df[bal].apply(lambda x: abc(x))
except:
    print("\nBalance Column Missing")

In [ ]:
df["flag"]=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

In [ ]:
df=df.T.drop_duplicates().T
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
df["flag2"]=np.arange(len(df))

df.head()

In [ ]:
df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)
df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

In [ ]:
df.head()

In [ ]:
df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
df.head()

In [ ]:
df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
df.to_csv("parsed.csv",index=0)

In [ ]:
try:
    narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
    print("try1")
except:
    try:
        narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
    except:
        try:
            narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
        except:
            pass

In [ ]:
df.reset_index(drop=True,inplace=True)
df = df[pd.notnull(df[narr])]
df.head()

In [ ]:
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass
df.head()

In [ ]:
# df.replace(to_replace = '', value =np.nan,inplace=True)

In [ ]:
# try:
#     narr=[n for n in df.columns if "PARTICULARS" in str(n).upper()][0]
#     print("try1")
# except:
#     try:
#         narr=[n for n in df.columns if "NARRATION" in str(n).upper()][0]
#     except:
#         try:
#             narr=[n for n in df.columns if "DESCRIPTION" in str(n).upper()][0] 
#         except:
#             pass

In [ ]:
# df.reset_index(drop=True,inplace=True)
# df = df[pd.notnull(df[narr])]
# df.head()

In [ ]:
df.iloc[:,-1]=df.iloc[:,-1].fillna(df.iloc[:,-2])

In [ ]:
def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
    df[bal]=df[bal].apply(lambda x: abc(x))
except:
    print("\nBalance Column Missing")

In [ ]:
df["flag"]=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

In [ ]:
df.head()

In [ ]:
df.to_csv("dt2.csv",index=0)

In [ ]:
df=df.T.drop_duplicates().T
try:
    df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("(Cr)","").replace("(Dr)","")).astype(float)
except:pass
df["flag"]=df.iloc[:,0].astype(str)+df[bal].astype(str)
df["flag2"]=np.arange(len(df))
df.head()

In [ ]:
# df.replace(to_replace = '', value =np.nan,inplace=True)

In [ ]:
df.head()

In [ ]:
df.loc[df[["flag"]].duplicated(keep=False), 'flag'] = df["flag"]+df["flag2"].astype(str)


In [ ]:
df.head()

In [ ]:
df['flag']=df['flag'].apply(lambda row : np.nan if 'nannan' in row else row).fillna(method='bfill')

In [ ]:
df.head()

In [ ]:
for i in df["Particulars"]:
    print(i,";",type(i))

In [ ]:
df[narr]=df.groupby('flag')[narr].transform(lambda x:''.join(x))
df=df.drop_duplicates(['flag'],keep='last').iloc[0:,:-2].reset_index(drop=True)
df.to_csv("parsed.csv",index=0)